# AtomsBase integration

[AtomsBase.jl](https://github.com/JuliaMolSim/AtomsBase.jl) is a common interface
for representing atomic structures in Julia. DFTK directly supports using such
structures to run a calculation as is demonstrated here.

In [1]:
using DFTK

## Feeding an AtomsBase AbstractSystem to DFTK
In this example we construct a silicon system using the `ase.build.bulk` routine
from the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
(ASE), which is exposed by [ASEconvert](https://github.com/mfherbst/ASEconvert.jl)
as an AtomsBase `AbstractSystem`.

In [2]:
# Construct bulk system and convert to an AbstractSystem
using ASEconvert
system_ase = ase.build.bulk("Si")
system = pyconvert(AbstractSystem, system_ase)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0    2.715    2.715;
                            2.715        0    2.715;
                            2.715    2.715        0]u"Å"

    Atom(Si, [       0,        0,        0]u"Å")
    Atom(Si, [  1.3575,   1.3575,   1.3575]u"Å")


To use an AbstractSystem in DFTK, we attach pseudopotentials, construct a DFT model,
discretise and solve:

In [3]:
system = attach_psp(system; Si="hgh/lda/si-q4")

model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921673507401                   -0.69    6.1         
  2   -7.926164116341       -2.35       -1.22    1.0    206ms
  3   -7.926837271540       -3.17       -2.37    1.8    232ms
  4   -7.926861526865       -4.62       -3.05    2.9    290ms
  5   -7.926861647554       -6.92       -3.43    1.8    212ms
  6   -7.926861671303       -7.62       -3.82    2.0    215ms
  7   -7.926861680334       -8.04       -4.31    1.4    200ms
  8   -7.926861681706       -8.86       -4.77    2.0    246ms
  9   -7.926861681855       -9.83       -5.19    1.9    214ms
 10   -7.926861681870      -10.83       -5.70    1.8    213ms
 11   -7.926861681873      -11.58       -6.74    1.8    212ms
 12   -7.926861681873      -13.02       -7.41    2.8    265ms
 13   -7.926861681873   +  -15.05       -7.76    2.1    223ms
 14   -7.926861681873      -14.35       -8.56    1.9    212ms


If we did not want to use ASE we could of course use any other package
which yields an AbstractSystem object. This includes:

### Reading a system using AtomsIO

In [4]:
using AtomsIO

# Read a file using [AtomsIO](https://github.com/mfherbst/AtomsIO.jl),
# which directly yields an AbstractSystem.
system = load_system("Si.extxyz")

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921690659112                   -0.69    5.6         
  2   -7.926168164086       -2.35       -1.22    1.0    281ms
  3   -7.926837588103       -3.17       -2.37    1.8    226ms
  4   -7.926861517825       -4.62       -3.04    2.9    243ms
  5   -7.926861646043       -6.89       -3.41    1.9    213ms
  6   -7.926861670785       -7.61       -3.82    1.8    250ms
  7   -7.926861680280       -8.02       -4.29    1.4    198ms
  8   -7.926861681721       -8.84       -4.79    1.9    211ms
  9   -7.926861681856       -9.87       -5.23    1.8    209ms
 10   -7.926861681871      -10.83       -5.79    1.9    248ms
 11   -7.926861681873      -11.73       -6.97    2.1    223ms
 12   -7.926861681873      -13.42       -7.45    3.5    263ms
 13   -7.926861681873      -14.75       -8.47    1.6    210ms


The same could be achieved using [ExtXYZ](https://github.com/libAtoms/ExtXYZ.jl)
by `system = Atoms(read_frame("Si.extxyz"))`,
since the `ExtXYZ.Atoms` object is directly AtomsBase-compatible.

### Directly setting up a system in AtomsBase

In [5]:
using AtomsBase
using Unitful
using UnitfulAtomic

# Construct a system in the AtomsBase world
a = 10.26u"bohr"  # Silicon lattice constant
lattice = a / 2 * [[0, 1, 1.],  # Lattice as vector of vectors
                   [1, 0, 1.],
                   [1, 1, 0.]]
atoms  = [:Si => ones(3)/8, :Si => -ones(3)/8]
system = periodic_system(atoms, lattice; fractional=true)

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921675284925                   -0.69    5.8         
  2   -7.926167022739       -2.35       -1.22    1.0    215ms
  3   -7.926839925736       -3.17       -2.37    1.8    245ms
  4   -7.926864927191       -4.60       -3.01    2.5    265ms
  5   -7.926865041662       -6.94       -3.34    1.6    215ms
  6   -7.926865077411       -7.45       -3.72    1.9    262ms
  7   -7.926865091625       -7.85       -4.37    1.1    197ms


## Obtaining an AbstractSystem from DFTK data

At any point we can also get back the DFTK model as an
AtomsBase-compatible `AbstractSystem`:

In [6]:
second_system = atomic_system(model)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0     5.13     5.13;
                             5.13        0     5.13;
                             5.13     5.13        0]u"a₀"

    Atom(Si, [  1.2825,   1.2825,   1.2825]u"a₀")
    Atom(Si, [ -1.2825,  -1.2825,  -1.2825]u"a₀")


Similarly DFTK offers a method to the `atomic_system` and `periodic_system` functions
(from AtomsBase), which enable a seamless conversion of the usual data structures for
setting up DFTK calculations into an `AbstractSystem`:

In [7]:
lattice = 5.431u"Å" / 2 * [[0 1 1.];
                           [1 0 1.];
                           [1 1 0.]];
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

third_system = atomic_system(lattice, atoms, positions)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0  5.13155  5.13155;
                          5.13155        0  5.13155;
                          5.13155  5.13155        0]u"a₀"

    Atom(Si, [ 1.28289,  1.28289,  1.28289]u"a₀")
    Atom(Si, [-1.28289, -1.28289, -1.28289]u"a₀")
